# Regions with Convolutional Neural Networks model for image segmentation:

In [ ]:
import Mask_RCNN.functions as f
import Mask_RCNN.utils as utils

from Mask_RCNN.bowl_config import bowl_config
from Mask_RCNN.bowl_dataset import BowlDataset
from Mask_RCNN.model import log
from Mask_RCNN.inference_config import inference_config
from Mask_RCNN.utils import rle_encode, rle_decode, rle_to_string

from Mask_RCNN import model as modellib
from Mask_RCNN.bowl_config import bowl_config
from Mask_RCNN.bowl_dataset import BowlDataset
from Mask_RCNN.model import log

In [ ]:
import pandas as pd
import cv2
import numpy as np

from tqdm import tqdm

In [ ]:
import os
import time

from glob import glob

## Root directory of the project:

In [ ]:
ROOT_DIR = os.getcwd()

## Directory to save logs and trained model:

In [ ]:
MODEL_DIR = os.path.join(ROOT_DIR, "model/checkpoint")

In [ ]:
gen_predictions = True
train_model = False

if gen_predictions:
    model = modellib.MaskRCNN(mode="inference", 
                              config=inference_config,
                              model_dir=MODEL_DIR)
elif train_model:
    model = modellib.MaskRCNN(mode="training", 
                              config=bowl_config,
                              model_dir=MODEL_DIR)

In [ ]:
CHECKPOINT_PATH = os.path.join(MODEL_DIR, 'mask_rcnn_bowl_0001.h5')

In [ ]:
if train_model !=False:
    # Training dataset
    dataset_train = BowlDataset()
    dataset_train.load_bowl('./fixed_data/train')
    dataset_train.prepare()

In [ ]:
if train_model != False:
    # Validation dataset
    dataset_val = BowlDataset()
    dataset_val.load_bowl('./fixed_data/train')
    dataset_val.prepare()

In [ ]:
TEST_DIR = './test_2/'

In [ ]:
if gen_predictions:
    # Test dataset
    dataset_test = BowlDataset()
    dataset_test.load_bowl(TEST_DIR)
    dataset_test.prepare()

### Build RCNN model:

In [ ]:
if train_model !=False:
    # Which weights to start with?
    init_with = "checkpoint"  # imagenet, coco, checkpoint or last
else:
    init_with = "last" # last or checkpoint

In [ ]:
if train_model !=False:
    if init_with == "imagenet":
        model.load_weights(model.get_imagenet_weights(), by_name=True)
    elif init_with == "coco":
        # Load weights trained on MS COCO, but skip layers that
        # are different due to the different number of classes
        # See README for instructions to download the COCO weights
        model.load_weights(COCO_MODEL_PATH, by_name=True,
                           exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                    "mrcnn_bbox", "mrcnn_mask"])
    elif init_with == "checkpoint":
        model.load_weights(CHECKPOINT_PATH, by_name=False)
    elif init_with == "last":
        # Load the last model you trained and continue training
        model.load_weights(model.find_last()[1], by_name=True)
    
    model.train(dataset_train,dataset_val, 
                learning_rate=bowl_config.LEARNING_RATE / 10,
                epochs=2, 
                layers="all")
elif gen_predictions:
    if init_with == "checkpoint":
            model.load_weights(CHECKPOINT_PATH, by_name=False)
    else:
        # Recreate the model in inference mode
        model = modellib.MaskRCNN(mode="inference", 
                              config=inference_config,
                              model_dir=MODEL_DIR)
        model_path = model.find_last()[1]
        # Load trained weights (fill in path to trained weights here)
        assert model_path != "", "Provide path to trained weights"
        print("Loading weights from ", model_path)
        model.load_weights(model_path, by_name=True)

In [ ]:
SUBMISSION_FILE = './stage2_sample_submission_final.csv'

def generate_RCNN_prediction(model = None, \
                             SUBMISSION_FILE = None, \
                             TEST_DIR = None, \
                             OUTPUT_FILE = None):
    
    output = []
    sample_submission = pd.read_csv(SUBMISSION_FILE)
    
    ImageId = []
    EncodedPixels = []
    
    for image_id in tqdm(sample_submission.ImageId):
        image_path = os.path.join(TEST_DIR, image_id, 'images', image_id + '.png')
    
        original_image = cv2.imread(image_path)
        results = model.detect([original_image], verbose=0)
        r = results[0]
    
        masks = r['masks']
        
        ImageId_batch, EncodedPixels_batch = f.numpy2encoding_no_overlap2(masks, image_id, r['scores'])
        ImageId += ImageId_batch
        EncodedPixels += EncodedPixels_batch
    
    f.write2csv(OUTPUT_FILE, ImageId, EncodedPixels)

In [ ]:
def generate_timestamp():
    timestring = time.strftime("%Y_%m_%d-%H_%M_%S")
    print ("Time stamp generated: "+timestring)
    return timestring

In [ ]:
timestr = generate_timestamp()

In [ ]:
generate_RCNN_prediction(model = model,
                         SUBMISSION_FILE = SUBMISSION_FILE,
                         TEST_DIR = TEST_DIR,
                         OUTPUT_FILE = os.path.join('./sub-dsbowl2018-RCNN_'+ timestr + '.csv'))